In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')


In [ ]:
train_features.head()

In [ ]:
# assuming train_features and train_targets follows same index order

In [ ]:
train = train_features
target = train_targets
test = test_features

In [ ]:
train.cp_type = train.cp_type.map({'trt_cp':1,'ctl_vehicle':0})
train.cp_dose = train.cp_dose.map({'D1':0,'D2':1})

test.cp_type = test.cp_type.map({'trt_cp':1,'ctl_vehicle':0})
test.cp_dose = test.cp_dose.map({'D1':0,'D2':1})

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(train.drop('sig_id',axis=1),target.drop('sig_id',axis=1),test_size=.2)

In [ ]:
X=train.drop('sig_id',axis=1)
y=target.drop('sig_id',axis=1)

In [ ]:
import lightgbm

In [ ]:
cat_cols = ['cp_type','cp_time','cp_dose']
categorical_features  =[]
for cols in cat_cols:
    categorical_features.append(xtrain.columns.get_loc(cols))
print(categorical_features)

In [ ]:
train_data = lightgbm.Dataset(xtrain, label=ytrain, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(xtest, label=ytest)

In [ ]:
y.columns

In [ ]:
columns = y.columns
for col in columns:
    print("Column id ", col)
    train_data = lightgbm.Dataset(X, label=y[col], categorical_feature=categorical_features)
    test_data = lightgbm.Dataset(X, label=y[col])
    parameters = {
        'application': 'binary',
        'objective': 'binary',
        'metric': 'auc',
        'is_unbalance': 'true',
        'boosting': 'gbdt',
        'max_leaves': 350,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.6,
        'bagging_freq': 20,
        'learning_rate': 0.005,
        'verbose': 0
    }

    model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=test_data,
                           num_boost_round=500,
                           early_stopping_rounds=10)
    submission[col] = model.predict(test.drop('sig_id',axis=1))

In [ ]:
model.predict(test.drop('sig_id',axis=1))

In [ ]:
 submission.to_csv("submission.csv",index=False)